In [1]:
def add_evaluation_step(result_tensor):
  """Inserts the operations we need to evaluate the accuracy of our results.

  Args:
    result_tensor: The new final node that produces results.
    ground_truth_tensor: The node we feed ground truth data
    into.

  Returns:
    Tuple of (evaluation step, prediction).
  """
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
  return  prediction

In [5]:
def load_graph(file_name):
  with open(file_name,'rb') as f:
    content = f.read()
  graph_def = tf.GraphDef()
  graph_def.ParseFromString(content)
  with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')
  return graph

In [113]:
#!/usr/bin/python
#coding:utf-8
#
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#此代码是从evaluate.py改造而来


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import sys
import argparse

import numpy as np
import PIL.Image as Image
import tensorflow as tf 


import os
import time

def evaluate_graph(graph_file_name):
    with load_graph(graph_file_name).as_default() as graph:
        
        image_buffer_input = graph.get_tensor_by_name('input:0')#第0个输出
        final_tensor = graph.get_tensor_by_name('final_result:0')
        accuracy = add_evaluation_step(final_tensor)# 返回的是argmax的结果，1是猫，0是狗
        
        logits = graph.get_tensor_by_name("final_training_ops/Wx_plus_b/add:0")
        
    image_dir = '/root/tensorflow-for-poets-dogs_and_cats/tf_files/dogs_and_cats_photos/test1'
#     image_dir = '/root/tensorflow-for-poets-dogs_and_cats/tf_files/dogs_and_cats_photos/train/dogs/'

    
    
    
    accuracies = []
    time1 = time.time()
    with tf.Session(graph=graph) as sess:
        summaries_dir = '/tmp/summaries'
        writer = tf.summary.FileWriter(summaries_dir + '/evaluate',
                                          sess.graph)
        
        for filename in range(1,125):    
            image = Image.open(os.path.join(image_dir, str(filename) + '.jpg')).resize((224,224),Image.ANTIALIAS)
            image = np.array(image, dtype=np.float32)[None,...]
            image = (image-128)/128.0

            feed_dict={
                image_buffer_input: image}

            eval_accuracy = sess.run([accuracy], feed_dict)
            
#             writer.add_graph(sess.graph)
            accuracies.append(eval_accuracy)
            
            if (filename % 50 == 0) :print('step:',filename,'\t total time:', time.time() - time1, 's')
    return np.squeeze(accuracies)





In [114]:

if __name__ == "__main__":
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    results = evaluate_graph('/root/tensorflow-for-poets-dogs_and_cats//tf_files/optimized_graph.pb')
    import pandas as pd
    submit = pd.DataFrame({'id':list(range(1,125)),'label':results})
    submit.to_csv('submit_test.csv', index=False, doublequote=False)


    
  

step: 50 	 total time: 3.606268882751465 s
step: 100 	 total time: 7.355381488800049 s


In [109]:
    print(submit['label'].value_counts())

0    1217
1      32
Name: label, dtype: int64
